# Entry G15: Global Density Comparison

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
%matplotlib inline

import warnings
warnings.filterwarnings('once')

from neo4j import GraphDatabase

In [3]:
uri = "bolt://localhost:7687"

driver = GraphDatabase.driver(uri, auth=('neo4j', 'password'))

uni_session = driver.session(database="unimodal")
bi_session = driver.session(database="bimodal")
mix_session = driver.session(database="mixmodal")

//anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Number of possible relationships

In [22]:
node_cts = pd.DataFrame(uni_session.run("call apoc.meta.stats() YIELD labels").value()).rename({0:'unimodal'}).append(
pd.DataFrame(bi_session.run("call apoc.meta.stats() YIELD labels").value()).rename({0:'bimodal'})).append(
pd.DataFrame(mix_session.run("call apoc.meta.stats() YIELD labels").value()).rename({0:'mixmodal'})).fillna(0)

node_cts['All_nodes'] = node_cts['Hero'] + node_cts['Comic']

node_cts = node_cts.merge(pd.DataFrame(index=['unimodal', 'bimodal', 'mixmodal'], data={'possible_rels':
                                                               [(node_cts.loc['unimodal', 'All_nodes'] * (node_cts.loc['unimodal', 'All_nodes']  - 1))/2,
                                                               node_cts.loc['bimodal', 'Hero'] * node_cts.loc['bimodal', 'Comic'],
                                                               (node_cts.loc['mixmodal', 'All_nodes'] * (node_cts.loc['mixmodal', 'All_nodes']  - 1))/2]}), 
               left_index=True, right_index=True)

node_cts = node_cts.merge(
pd.DataFrame(uni_session.run('''MATCH ()-[r]->()
RETURN count(r) as actual_rels''').data()).rename({0:'unimodal'}).append(
pd.DataFrame(bi_session.run('''MATCH ()-[r]->()
RETURN count(r) as actual_rels''').data()).rename({0:'bimodal'})).append(
pd.DataFrame(mix_session.run('''MATCH ()-[r]->()
RETURN count(r) as actual_rels''').data()).rename({0:'mixmodal'})), left_index=True, right_index=True)

node_cts

,Hero,Comic,All_nodes,possible_rels,actual_rels
unimodal,6439,0.0,6439.0,20727141.0,171644
bimodal,6439,12651.0,19090.0,81459789.0,96104
mixmodal,6439,12651.0,19090.0,182204505.0,267748


## Global density

In [23]:
node_cts['global_density'] = node_cts['actual_rels'] / node_cts['possible_rels']

node_cts

//anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Hero,Comic,All_nodes,possible_rels,actual_rels,global_density
unimodal,6439,0.0,6439.0,20727141.0,171644,0.008281
bimodal,6439,12651.0,19090.0,81459789.0,96104,0.001180
mixmodal,6439,12651.0,19090.0,182204505.0,267748,0.001469
